# Project 8 - Stock Price Prediction - Data Extracting, Cleaning and EDA

- Course Name :         Applied Machine Learning
- Course instructor:    Sohail Tehranipour
- Student Name :        Afshin Masoudi Ashtiani
- Project 8 :           Stock Price Prediction
- Date :                September 2024
- Files :               1/2

## Step 1 : Install required libraries



In [52]:
%pip install numpy pandas
%pip install yfinance plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Step 2 : Import required libraries

In [53]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

# Mount google drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# Warnings
import warnings
warnings.filterwarnings('ignore')

## Step 3 : Load and prepare the dataset (Stock Price)


In [54]:
def fetch_stock_price_data(ticker: str, start: str = "2019-01-01", end: str = "2024-01-01") -> pd.DataFrame:
    # Fetch data using yfinance
    data = yf.download(ticker, start, end)

    # Ensure the DataFrame is not empty
    if data.empty:
        raise ValueError(f"No data found for ticker: {ticker} between {start} and {end}")

    # Select necessary columns and reset index
    stock_data = pd.DataFrame()
    stock_data['Date'] = data.iloc[:,0].index.tolist()
    stock_data['Open'] = data.iloc[:,0].values.tolist()
    stock_data['High'] = data.iloc[:,1].values.tolist()
    stock_data['Low'] = data.iloc[:,2].values.tolist()
    stock_data['Close'] = data.iloc[:,3].values.tolist()

    # Convert Date to string format if necessary (pandas should handle this well)
    stock_data['Date'] = stock_data['Date'].dt.strftime('%Y-%m-%d')

    return stock_data

In [55]:
# Download historical stock data
ticker_symbols = ('AAPL', 'GOOG', 'TSLA')
ticker = ticker_symbols[2]

start = "2015-01-01"
end = "2025-01-01"
stock_data = fetch_stock_price_data(ticker, start, end)
# Display a summary of the fetched data
print(f"> Summary of {ticker} stock price data:")
stock_data.info()
stock_data.sample(5)

[*********************100%***********************]  1 of 1 completed

> Summary of TSLA stock price data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2516 non-null   object 
 1   Open    2516 non-null   float64
 2   High    2516 non-null   float64
 3   Low     2516 non-null   float64
 4   Close   2516 non-null   float64
dtypes: float64(4), object(1)
memory usage: 98.4+ KB


,Date,Open,High,Low,Close
739,2017-12-07,20.749332,20.749332,21.242001,20.736668
84,2015-05-05,15.530000,15.530000,15.966667,15.275333
2448,2024-09-25,257.019989,257.019989,257.049988,252.279999
1789,2022-02-09,310.666656,310.666656,315.423340,306.666656
1122,2019-06-19,15.095333,15.095333,15.184667,14.737333


## Step 4 : Data Cleaning

### I : Handle **missing values**

In [56]:
stock_data.isnull().sum()

Date     0
Open     0
High     0
Low      0
Close    0
dtype: int64

In [57]:
print(f'> Shape of {ticker} Train dataset before handling missing values: {stock_data.shape}')
stock_data.dropna(inplace=True)
print(f'> Shape of {ticker} Train dataset after handling missing values: {stock_data.shape}')

> Shape of TSLA Train dataset before handling missing values: (2516, 5)
> Shape of TSLA Train dataset after handling missing values: (2516, 5)


### II : Handle **duplicates**

In [58]:
stock_data.duplicated().sum()

0

In [59]:
# Handle Duplicate Values
print(f'> Shape of {ticker} dataset before handling duplicates is {stock_data.shape}')
stock_data = stock_data.drop_duplicates(ignore_index= True)
print(f'> Shape of {ticker} dataset after handling duplicates is {stock_data.shape}')

> Shape of TSLA dataset before handling duplicates is (2516, 5)
> Shape of TSLA dataset after handling duplicates is (2516, 5)


### III : Handle **outliers**

In [60]:
stock_data.describe()

,Open,High,Low,Close
count,2516.000000,2516.000000,2516.000000,2516.000000
mean,115.679423,115.679423,118.250154,112.986545
std,114.226440,114.226440,116.887822,111.485181
min,9.578000,9.578000,10.331333,9.403333
25%,17.185167,17.185167,17.481667,16.889166
50%,28.505667,28.505667,28.806000,27.349999
75%,220.205002,220.205002,225.354996,215.336670
max,479.859985,479.859985,488.540009,457.510010


- Visualize the outliers

In [61]:
def display_outliers(data: pd.DataFrame, ticker: str) -> None:
    """Display box plots for each numerical column of the given DataFrame, excluding the first column."""
    
    # Ensure there are at least two columns to plot
    if data.shape[1] < 2:
        print("DataFrame must contain at least two columns to display outliers.")
        return

    # Extract columns for plotting, skipping the first column
    columns_to_plot = data.columns[1:]

    # Create box plots for each selected column
    for column in columns_to_plot:
        # Create a box plot
        fig = px.box(data, x=column, points=False)

        # Add titles and labels
        fig.update_layout(
            title=f'{column} - {ticker} Stock Prices',
            template='plotly'
        )

        # Show the figure
        fig.show()

In [62]:
display_outliers(stock_data, ticker)

- Using **IQR**

In [63]:
def remove_outliers_iqr(data:pd.DataFrame, column_name:str, threshold:float= 1.5) -> pd.DataFrame:
    """Remove outliers using IQR method."""
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    IQR = Q3 - Q1
    return data[~((data[column_name] < (Q1 - threshold * IQR)) | (data[column_name] > (Q3 + threshold * IQR)))]

In [64]:
# Example usage
print(f'> Shape of {ticker} dataset before handling outliers is {stock_data.shape}.')
stock_data = remove_outliers_iqr(stock_data, 'Open')
print(f'> Shape of {ticker} dataset after  handling Open   outliers is {stock_data.shape}.')
stock_data = remove_outliers_iqr(stock_data, 'High')
print(f'> Shape of {ticker} dataset after  handling High   outliers is {stock_data.shape}.')
stock_data = remove_outliers_iqr(stock_data, 'Low')
print(f'> Shape of {ticker} dataset after  handling Low    outliers is {stock_data.shape}.')
stock_data = remove_outliers_iqr(stock_data, 'Close')
print(f'> Shape of {ticker} dataset after  handling Close  outliers is {stock_data.shape}.')
stock_data.reset_index(drop= True, inplace= True)
print(f'> Cleaned {ticker} Stock Price Data using IQR: ')
stock_data.info()
stock_data.sample(5)

> Shape of TSLA dataset before handling outliers is (2516, 5).
> Shape of TSLA dataset after  handling Open   outliers is (2516, 5).
> Shape of TSLA dataset after  handling High   outliers is (2516, 5).
> Shape of TSLA dataset after  handling Low    outliers is (2516, 5).
> Shape of TSLA dataset after  handling Close  outliers is (2516, 5).
> Cleaned TSLA Stock Price Data using IQR: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2516 non-null   object 
 1   Open    2516 non-null   float64
 2   High    2516 non-null   float64
 3   Low     2516 non-null   float64
 4   Close   2516 non-null   float64
dtypes: float64(4), object(1)
memory usage: 98.4+ KB


,Date,Open,High,Low,Close
1985,2022-11-18,180.190002,180.190002,185.190002,176.550003
1385,2020-07-06,91.438667,91.438667,91.852669,84.402664
831,2018-04-23,18.891333,18.891333,19.441334,18.822001
1321,2020-04-02,30.298000,30.298000,32.950668,29.760000
1357,2020-05-26,54.591331,54.591331,55.639999,54.380669


## Step 4 : Exploratory Data Analysis (EDA)

In [65]:
def plot_stock_price_data(data: pd.DataFrame, stock_name: str=None) -> None:
    """Plot historical stock price data including Open, High, Low, and Close prices."""

    # Extract necessary columns
    dates = data['Date']
    open_prices = data['Open']
    high_prices = data['High']
    low_prices = data['Low']
    close_prices = data['Close']

    # Create a plotly graph object
    fig = go.Figure()

    # Add traces for Open, High, Low, and Close prices
    fig.add_trace(go.Scatter(x=dates, y=open_prices, mode='lines', name='Open Price', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=dates, y=high_prices, mode='lines', name='High Price', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=dates, y=low_prices, mode='lines', name='Low Price', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=dates, y=close_prices, mode='lines', name='Close Price', line=dict(color='orange')))

    # Add titles and labels
    fig.update_layout(
        title=f'{stock_name} Historical Stock Prices',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly',
        xaxis_rangeslider_visible=True
    )

    # Show the plot
    fig.show()

In [66]:
# Assuming you have a DataFrame `stock_data` with columns 'Date', 'Open', 'High', 'Low', 'Close'
plot_stock_price_data(stock_data, ticker)

## Step 5 : Save the cleaned dataset

In [67]:
# Saving stock data
def save_stock_data(data: pd.DataFrame, ticker: str, folder: str) -> None:
    """Save stock data to a CSV file."""
    # Create a safe file name based on the ticker symbol
    filename = f"{ticker}_stock_data.csv"
    data_path = os.path.join(folder, filename)

    # Save the DataFrame to CSV
    data.to_csv(data_path, index=False)
    print(f'> {ticker} Stock Price dataset saved to: {data_path}')

In [68]:
# Assuming `stock_data` is your DataFrame with stock information
folder_path = r'C:/Users/Afshin/Desktop/10_Projects/Project_8_Apple_Stock_Price_Prediction/datasets'
# folder_path = '/content/drive/My Drive/Applied Machine Learning/Project 8 : Stock Price Prediction/datasets

save_stock_data(stock_data, ticker, folder_path)

> TSLA Stock Price dataset saved to: C:/Users/Afshin/Desktop/10_Projects/Project_8_Apple_Stock_Price_Prediction/datasets\TSLA_stock_data.csv
